In [68]:
import json
import pandas as pd
import numpy as np
import re
import nltk
from langdetect import detect

In [2]:
json_data_path_66 = 'media/indeed_usa-indeed_usa_job_data__20211001_20211231_deduped_n_merged_20220305_004258919873466.ldjson'
json_data_path_88 = 'media/indeed_usa-indeed_usa_job_data__20211001_20211231_deduped_n_merged_20220305_004328202689288.ldjson'
engineering_jobs = 'media/engineering_jobs.csv'

In [20]:
data = pd.read_csv(engineering_jobs)

In [21]:
def read_json_data(file_path: str) -> pd.DataFrame:
    """Function for read json data and convert to pandas DataFrame"""
    list_data = []
    with open(file_path, mode='r', errors='ignore', encoding="utf8") as json_file:
        for ob in json_file:
            list_data.append(json.loads(ob))
    return pd.DataFrame(list_data)

In [22]:
indeed_66 = read_json_data(json_data_path_66)
indeed_88 = read_json_data(json_data_path_88)

In [23]:
data = data.replace(np.nan,'')
indeed_66 = indeed_66.replace(np.nan, '')
indeed_88 = indeed_88.replace(np.nan, '')

In [24]:
data['description'] = data['description']+data['basic_qualifications']+data['prefered_qualifications']+data['category']+data['skills']
indeed_66['job_description'] = indeed_66['job_description'] + indeed_66['category'] + indeed_66['company_description']
indeed_88['job_description'] = indeed_88['job_description'] + indeed_88['category'] + indeed_88['company_description']

In [25]:
sum(pd.read_csv(engineering_jobs)['title']=='')

0

## Data concatenation

In [26]:
data = data[['title', 'description']]

indeed = pd.concat([indeed_66[['job_title', 'job_description']], indeed_88[['job_title', 'job_description']]])
indeed = indeed.rename(columns={'job_title': 'title', 'job_description': 'description'})

In [27]:
data = pd.concat([data[['title', 'description']], indeed]).reset_index(drop=True)

In [28]:
data

,title,description
0,software development manager,You are an experienced hands-on manager with a...
1,software development engineer,Amazon is driven by being “the world’s most cu...
2,software development engineer,Have you ever wondered what it takes to build ...
3,quality assurance engineer,Fire TV client software and service technologi...
4,software development engineer,The Amazon Devices team designs and engineers ...
...,...,...
149276,Biomedical Technician 1 Chicago IL,Job Description Summary As the Biomed Technici...
149277,Marketing Assistant,"Sports Facilities Management, LLC- Cornerstone..."
149278,Core Supervisor,We are looking for a Core Supervisor 40 hours ...
149279,Stores Supervisor,We are LAUSD. We are at the forefront of innov...


In [29]:
data.title.value_counts()

data scientist                                  4016
software developer                              3585
software development engineer                   1077
Administrative Assistant                         380
junior software developer                        335
                                                ... 
Pharmacy Tech I - La Porte - Part Time             1
ICTS Supply Chain Risk Management Consultant       1
Solution Architect - Early Science                 1
Machine Operator - 3rd Shift - Hickman, KY         1
Stores Supervisor                                  1
Name: title, Length: 88689, dtype: int64

## Levenshtein

In [30]:
from Levenshtein import distance as levenshtein_distance


In [31]:
levenshtein_distance('data scientist', 'data sciences engineer')

11

In [32]:
levenshtein_distance('data scientist', 'customer experience data scientist, google cloud support')

42

In [33]:
levenshtein_distance('data scientist', 'hardware development engineer, home team')

32

In [34]:
title_data = data['title'].unique()

In [35]:
len(title_data)

88689

## Data filter

In [37]:
copy_data = data.copy()

In [38]:
JUNIOR_LIST = ["junior", "jr", "jr.", "intern", "entry level", "graduate", "associate", "assistant", "trainee",
"internship", "apprentice", "post grad", "post-grad", "grad", "recent graduate", "masters",
"under grad", "under graduate", "representative", ]

MID_LIST = ["lead", "leader", "leading", "leadership", "team lead", "team leader", "coordinator",
"account executive", "manager", "managing", "management", "supervisor", "middle"]

SENIOR_LIST = ["leader", "manager", "executive", "director", "coordinator", "administrator", "controller",
"officer", "organizer", "supervisor", "head", "overseer", "chief", "foreman", "principal", "president",
"lead", "senior", "sr.", "sr", "ceo", "coo", "cfo", "cio", "cto", "cmo", "chro", "cdo", "cpo", "cco",
"cio", "principle", "partner", "vp", "vice principle", "vice president", "vp", "counsel", "counselor", ]

In [39]:
filters = '|'.join(JUNIOR_LIST + MID_LIST + SENIOR_LIST)

In [40]:
copy_data['position'] = ''
for option in JUNIOR_LIST+MID_LIST+SENIOR_LIST:
    print(f'{option}|{option.capitalize()}')
    copy_data.loc[copy_data['title'].str.contains(option, regex=False, case=False), 'position']  = option
    
copy_data['title'] = copy_data['title'].str.replace(filters, "", case=False)

junior|Junior
jr|Jr
jr.|Jr.
intern|Intern
entry level|Entry level
graduate|Graduate
associate|Associate
assistant|Assistant
trainee|Trainee
internship|Internship
apprentice|Apprentice
post grad|Post grad
post-grad|Post-grad
grad|Grad
recent graduate|Recent graduate
masters|Masters
under grad|Under grad
under graduate|Under graduate
representative|Representative
lead|Lead
leader|Leader
leading|Leading
leadership|Leadership
team lead|Team lead
team leader|Team leader
coordinator|Coordinator
account executive|Account executive
manager|Manager
managing|Managing
management|Management
supervisor|Supervisor
middle|Middle
leader|Leader
manager|Manager
executive|Executive
director|Director
coordinator|Coordinator
administrator|Administrator
controller|Controller
officer|Officer
organizer|Organizer
supervisor|Supervisor
head|Head
overseer|Overseer
chief|Chief
foreman|Foreman
principal|Principal
president|President
lead|Lead
senior|Senior
sr.|Sr.
sr|Sr
ceo|Ceo
coo|Coo
cfo|Cfo
cio|Cio
cto|Cto
cmo|

C:\Users\annch\AppData\Local\Temp\ipykernel_1104\653939572.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  copy_data['title'] = copy_data['title'].str.replace(filters, "", case=False)


In [41]:
copy_data['title'] = copy_data['title'].apply(str.lower).apply(str.strip)


In [42]:
save_data = copy_data.copy()

In [44]:
sum(data['title']=='')

0

In [46]:
# save current result
for id, df_i in  enumerate(np.array_split(save_data.sort_values(by=['title']).reset_index(drop=True), 2)):
    df_i.to_excel(f'media/output/review_{id}.xlsx', index=False)

C:\Users\annch\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://eatbarenaked.com/%20Prep%20Cook:%20Seeking%20an%20experienced,%20organized,%20and%20efficient%20prep%20cook%20that%20will%20uphold%20our%20standards%20of%20quality%20and%20excellence.%20We%20have%20a%20varied%20menu%20with%20multiple%20ingredients%20and%20will%20use%20multiple%20cooking%20methods.%20Line%20Cook/Food%20Producer:%20Candidate%20should%20have%20a%20general%20scope%20in%20culinary,%20will%20build%20menu%20items%20and%20uphold%20all%20food%20safety%20and%20sanitation%20standards%20to%20a%20premium.%20Job%20Types:%20Full-time,%20Part-time%20Pay:%20$8.00%20-%20$15.00%20per%20hour%20Guest%20Service%20Attendant:%20Our%20guest%20ambassador,%20will%20attend%20to%20guests’%20needs%20and%20continued%20needs,%20continued%20service%20in%20the%20dining%20room%20by%20suggestively%20selling%20our%20great%20offerings.%20Cashier/Sales:%20Will%20guide%20guests%20through%20our%20menu%2

C:\Users\annch\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://www.tendonllc.com/construction-worker%20There%20is%20potential%20to%20be%20hired%20on%20permanently%20after%20our%20project%20in%20Myrtle%20Beach%20is%20complete.%20Please%20apply%20today!%20DESCRIPTION:%20Construction/%20concrete%20laborer%20will:%20Load%20and%20unload%20construction%20materials%20(concrete,%20cable%20spools,%20etc)%20Construction/%20concrete%20laborer%20will:%20Break%20up%20concrete,%20bag%20concrete,%20and%20dispose%20of%20concrete%20Construction/%20concrete%20laborer%20will:%20Follow%20instructions%20from%20construction%20project%20leaders%20and%20construction%20project%20managers/%20coordinators%20Construction/%20concrete%20laborer%20will:%20Use%20hand%20tools%20and%20industrial%20construction%20machinery/%20heavy%20equipment%20Construction/%20concrete%20laborer%20will:%20Pour%20concrete%20Construction/%20concrete%20laborer%20will:%20Help%20equipment%20opera

C:\Users\annch\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://jobs.kroger.com/job/Sheboygan-Deli-Lead-432-WI-53081/778583700/%20Please%20apply%20at%20the%20above%20link%20for%20our%20Kroger%20website%20for%20our%20online%20application.%20__________________________________________________________________________________________%20We%20are%20looking%20for%20a%20courteous,%20customer-focused%20Deli%20Clerk%20who%20is%20committed%20to%20providing%20customers%20with%20the%20best%20service%20possible.%20Deli%20Clerks%20will%20clean%20and%20prepare%20workstations%20and%20displays,%20answer%20questions%20and%20respond%20to%20customer%20requests,%20and%20provide%20information%20to%20clients%20to%20ensure%20that%20they%20select%20the%20products%20that%20best%20fit%20their%20needs.%20To%20succeed%20as%20a%20Deli%20Clerk,%20you%20should%20be%20polite,%20prompt,%20and%20knowledgeable.%20Additionally,%20you%20should%20be%20able%20to%20meet%20the%20physic

C:\Users\annch\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://kroger.eightfold.ai/careers?division=534%20-%20roundy%27s%20wisconsin%20%28pick%20%27n%20save%29&pid=9574848&location=sheboygan%2C%20wisconsin&query=Kroger%20Retail%20Clerk&domain=kroger.com&triggerGoButton=true Please apply at the above link for our Kroger website for our online application. What Do Meat Clerks Do? Meat clerks are responsible for a variety of tasks pertaining to the sale of meat, either at a butcher or a meat section of a grocery store. These tasks include providing quality customer service, weighing and wrapping customers' orders, cutting meat to customers' exact specifications, and maintaining a high product knowledge to most effectively facilitate customer orders and respond to queries. Aside from serving customers directly, meat clerks also have responsibilities behind the scenes, including: placing and receiving orders of a necessary quantity; unloading and

C:\Users\annch\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1170: UserWarning: Ignoring URL 'https://www.gapinc.com/en-us/jobs/w46/46/seasonal-sales-associate-bridgeport-village%20We’re%20hiring%20for%20the%20holidays!%20Join%20our%20team%20and%20enjoy%20a%2050%25%20discount%20at%20our%20family%20of%20brands.*%20A%20few%20other%20things%20to%20look%20forward%20to:%20part%20time%20benefits%20(health,%20dental%20&%20vision%20plans),%20child%20&%20family%20care%20support,%20mental%20health%20&%20well-being%20programs,%20volunteer%20opportunities,%20commuter%20benefits%20&%20community%20discounts.%20*Applicable%20to%20regular-priced%20merchandise%20at%20Gap,%20Banana%20Republic,%20Old%20Navy%20and%20Athleta,%20and%2030%25%20off%20at%20Outlet.%20About%20Gap%20Gap%20has%20always%20been%20about%20optimistic%20American%20style.%20Starting%20with%20our%20very%20first%20store%20on%20Ocean%20Avenue%20in%20San%20Francisco,%20opened%20almost%2050%20years%20ago%20by%20Doris%20and%20Don%20Fish

## Update Title Column Values

In [50]:
job_titles = pd.read_excel("media/job-lists.xlsx")
job_titles['job_title'] = job_titles['job_title'].apply(str.lower)
job_titles = job_titles["job_title"].values.tolist()

In [49]:
target_data = pd.read_excel("media/output/review_0.xlsx")
target_data = target_data[~target_data["title"].isnull()]
target_data["title"] = target_data["title"].apply(str.lower)

In [51]:
target_data_1 = pd.read_excel("media/output/review_1.xlsx")
target_data_1 = target_data_1[~target_data_1["title"].isnull()]
target_data_1["title"] = target_data_1["title"].apply(str.lower)

In [52]:
def update_title(_data):
    """Function for matching title with job-list values"""
    _data["found_positions"] = ""
    for job_title in job_titles:
        indices = _data[_data["title"].str.contains(job_title, case=False, regex=False)].index
        if indices.empty:
            continue
        _data.loc[indices, "found_positions"] = _data.loc[indices, "found_positions"] + f",{job_title}"
    return _data

In [53]:
def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", str(text).lower().strip())
     
    ## Tokenize (convert from string to list)
    lst_text = text.split()
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
            
    ## back to string from list
    text = " ".join(lst_text)
    return text

In [55]:
target_data["title"] = target_data["title"].apply(lambda x: utils_preprocess_text(x, flg_stemm=True, flg_lemm=True))
target_data

,title,description,position
1338,,"Think fresh, fast and fun (and delicious)! Thi...",manager
1339,full time daytim clean janitori,$1000 IN SIGN-ON BONUSES. AUTOMATIC RAISES. PA...,NaN
1340,immedi interview kroger pharmaci technician 23...,We are looking for a dependable Pharmacy Techn...,NaN
1341,incent offer of nur,Assisting the Director of Nursing with impleme...,cto
1342,now hire,Your opportunity to thrive in a workplace envi...,manager
...,...,...,...
74636,lea,Leasing Associate for Affordable Housing Apart...,associate
74637,lea,Position Description This individual shall hav...,associate
74638,lea,Senior Sales Expert Excellent Pegasus Benefits...,manager
74639,lea,Who We Are: Peak Property Group specializes in...,associate


In [56]:
target_data_1["title"] = target_data_1["title"].apply(lambda x: utils_preprocess_text(x, flg_stemm=True, flg_lemm=True))
target_data_1

,title,description,position
0,lea marqui at cinco ranch,Overview: Looking for something different? We ...,associate
1,lea assembl 118,Join Our Leasing Team! We have an exciting opp...,manager
2,lea breckinridg squar,"PMR Companies, a premier Multi -Family Propert...",associate
3,lea provid row,"Providence Row - Charlotte, NC: POSITION OVERV...",associate
4,lea axio 8400,"Avenue5 is growing, and we are in search of a ...",manager
...,...,...,...
74635,me o,デバイス、ドライバに加え、Chrome OS、Android などの Linux ベースシス...,chro
74636,brand design,we are currently seeking a lead brand designer...,lead
74637,mechan solidwork design,"Johnston Engineering, a growing product develo...",NaN
74638,fr,ab tasty (www.abtasty.com) is the customer exp...,cco


In [58]:
target_data = update_title(target_data)
target_data_1 = update_title(target_data_1)

In [59]:
target_data.shape, len(target_data.description.unique())

((73303, 4), 72543)

In [60]:
target_df = pd.concat([target_data, target_data_1])

In [61]:
target_df["found_positions"] = target_df["found_positions"].apply(lambda x: max(x.split(','), key=len)) # take the longest job titles 
target_df["found_positions"] = target_df["found_positions"].apply(lambda x: utils_preprocess_text(x, flg_stemm=True, flg_lemm=False))

In [62]:
target_df['found_positions'] = target_df['found_positions'].replace('', np.NaN)
target_df["title"] = target_df["found_positions"].fillna(target_df["title"])

In [63]:
target_df = target_df.drop_duplicates(subset='description')

In [64]:
target_df = target_df.drop(['position', 'found_positions'], axis=1)

In [65]:
len(target_df.description.unique())

146280

In [66]:
target_df

,title,description
1338,,"Think fresh, fast and fun (and delicious)! Thi..."
1339,janitor,$1000 IN SIGN-ON BONUSES. AUTOMATIC RAISES. PA...
1340,technician,We are looking for a dependable Pharmacy Techn...
1341,incent offer of nur,Assisting the Director of Nursing with impleme...
1342,now hire,Your opportunity to thrive in a workplace envi...
...,...,...
74635,me o,デバイス、ドライバに加え、Chrome OS、Android などの Linux ベースシス...
74636,brand design,we are currently seeking a lead brand designer...
74637,mechan solidwork design,"Johnston Engineering, a growing product develo..."
74638,fr,ab tasty (www.abtasty.com) is the customer exp...


In [67]:
# drop titles with less then 100 descriptions
title_counts = target_df.groupby('title').agg(count=("title", 'count')).reset_index().sort_values('count', ascending=False)
title_counts = title_counts[title_counts['count'] > 100]
target_df = target_df[target_df['title'].isin(title_counts['title'].values)]

In [ ]:
target_df = target_df.replace(r'\n|\t|@',' ', regex=True)
target_df = target_df[target_df['title']!='']

## language detection

In [ ]:
target_df = target_df.reset_index(drop=True)
target_df['title'] = target_df[target_df['title'].apply(lambda title: detect(title)=='en')]

# Save data

In [ ]:
target_df = target_df.reset_index(drop=True)
target_df.to_csv('data/training_all_data.csv',  sep="@", index=False)